In [ ]:
!apt-get install -y coinor-cbc

In [ ]:
import pandas as pd
from pyomo.environ import *
import matplotlib.pyplot as plt

# Load data files/sheets
file_path = 'Enter A matrix sheet address'
massbalance_df = pd.read_excel(file_path, sheet_name='A matrix', index_col=0).fillna(0)
price_df = pd.read_excel(file_path, sheet_name='Price_cost', header=None, index_col=0).fillna(0)
energy_df = pd.read_excel(file_path, sheet_name='Energy required', header=None, index_col=0).fillna(0)
f_df_70 = pd.read_excel(file_path, sheet_name='F_matrix_70', header=None, index_col=0).fillna(0)
f_df_80 = pd.read_excel(file_path, sheet_name='F_matrix_80', header=None, index_col=0).fillna(0)
f_df_90 = pd.read_excel(file_path, sheet_name='F_matrix_90', header=None, index_col=0).fillna(0)
f_df_95 = pd.read_excel(file_path, sheet_name='F_matrix_95', header=None, index_col=0).fillna(0)
f_df_100 = pd.read_excel(file_path, sheet_name='F_matrix_100', header=None, index_col=0).fillna(0)

# Define the model
model = ConcreteModel()

# Define the sets
model.p = Set(initialize=massbalance_df.columns.tolist())  # Processes
model.c = Set(initialize=massbalance_df.index.tolist())  # Chemicals
# a sets of lists for next stage use.
model.cA = Set(initialize=["C1"])
model.cB = Set(initialize=['C1','C2','C3','C4','C5','C6','C7','C8'])  # Input stream
model.cC = Set(initialize=['C9', 'C10', 'C11', 'C12'])                        #Desired output stream's elements, we can play around it.
model.pA = Set(initialize=['R38', 'R74', 'R9', 'R10', 'R30', 'R60', 'R69'])   #These recycling processes are not useful because they have zero values

# Initialize the parameters in dictionary format
model.Massbalance = Param(model.c, model.p, initialize=massbalance_df.stack().to_dict())
model.Price = Param(model.c, initialize=price_df.iloc[:, 0].to_dict())
model.Unitenergy = Param(model.p, initialize=energy_df.iloc[:, 0].to_dict())
model.F_70 = Param(model.c, initialize=f_df_70.iloc[:, 0].to_dict())
model.F_80 = Param(model.c, initialize=f_df_80.iloc[:, 0].to_dict())
model.F_90 = Param(model.c, initialize=f_df_90.iloc[:, 0].to_dict())
model.F_95 = Param(model.c, initialize=f_df_95.iloc[:, 0].to_dict())
model.F_100 = Param(model.c, initialize=f_df_100.iloc[:, 0].to_dict())

# Initialize variables
model.Scale = Var(model.p, within=NonNegativeReals, bounds=(0, 1), initialize=0)
model.Flow = Var(model.c, within=NonNegativeReals, bounds=(0, 1000000), initialize=0)

# Objective functions
def cost_rule(model):
    return sum(model.Flow[c] * model.Price[c] for c in model.c)

def energy_rule(model):
    return sum(model.Scale[p] * model.Unitenergy[p] for p in model.p)

# Constraints
def mass_balance_rule(model, c):
    return sum(model.Massbalance[c, p] * model.Scale[p] for p in model.p) == model.Flow[c]

Functional_unit = 1

def functional_unit_rule(model):
    return model.Scale['R0'] == Functional_unit

def all_external2(model, c):
    if c in model.cC:
        return model.Flow[c] >= model.F_95[c]
    return Constraint.Skip

def all_external(model, c):
    if c in model.cC:
        return model.Flow[c] <= model.F_100[c]
    return Constraint.Skip

def all_external1(model, c):
    if c in model.cB:
        return model.Flow[c] == 0
    return Constraint.Skip

def positive_flow_rule(model, c):
    return model.Flow[c] >= 0

def positive_scale_rule(model, p):
    return model.Scale[p] >= 0

def dummy(model, p):
    if p in model.pA:
        return model.Scale[p] == 0
    return Constraint.Skip


# Apply constraints
model.mass_balance_constraints = Constraint(model.c, rule=mass_balance_rule)
model.functional_unit_constraints = Constraint(rule=functional_unit_rule)
model.all_external_constraints = Constraint(model.c, rule=all_external)
model.all_external1_constraints = Constraint(model.c, rule=all_external1)
model.all_external1_constraints = Constraint(model.c, rule=all_external2)
model.positive_flow_constraints = Constraint(model.c, rule=positive_flow_rule)
model.positive_scale_constraints = Constraint(model.p, rule=positive_scale_rule)
model.dummy_constraints = Constraint(model.p, rule=dummy)

solver = SolverFactory('cbc')

def solve_and_get_values(obj_rule, sense=minimize):
    # Remove all existing objectives
    for obj in list(model.component_objects(Objective, active=True)):
        model.del_component(obj)

    # Define new objective
    model.obj = Objective(rule=obj_rule, sense=sense)

    # Solve the model
    solver.solve(model, tee=True)

    # Extract values
    obj_value = value(model.obj)
    cost = obj_value if obj_rule == cost_rule else value(cost_rule(model))
    energy = obj_value if obj_rule == energy_rule else value(energy_rule(model))

    flow_values = {c: model.Flow[c].value if model.Flow[c].value is not None else 0 for c in model.c}
    scale_values = {p: model.Scale[p].value if model.Scale[p].value is not None else 0 for p in model.p}

    return cost, energy, flow_values, scale_values

# Solve for Cost Minimization
cost_min, energy_at_cost_min, flow_cost_min, scale_cost_min = solve_and_get_values(cost_rule, sense=maximize)
print(f"Maximum profit: {cost_min}, Corresponding Energy: {energy_at_cost_min}")

model.display()

# Solve for Energy Minimization
cost_at_energy_min, energy_min, flow_emission_min, scale_emission_min = solve_and_get_values(energy_rule, sense=minimize)
print(f"Minimized Energy: {energy_min}, Corresponding Profit: {cost_at_energy_min}")

model.display()

# Plot Cost vs. Energy
plt.figure(figsize=(8, 6))
plt.scatter(cost_min, energy_at_cost_min, color='blue', label="Profit Maximization")
plt.scatter(cost_at_energy_min, energy_min, color='red', label="Energy Minimization")
plt.xlabel("Profit [$]")
plt.ylabel("Energy [kJ]")
plt.title("Performance of Profit vs. Energy Optimization")
plt.legend()
plt.grid(True)
save_path = "/content/example.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from pyomo.environ import *
import matplotlib.pyplot as plt

# Solve for minimum cost
min_cost, min_cost_emissions, _, _ = solve_and_get_values(cost_rule, sense=maximize)
# Solve for minimum emissions
min_emissions_cost, min_emissions, _, _ = solve_and_get_values(energy_rule, sense=minimize)

# *Epsilon constraint method for Min-Cost to Min-Emissions*
def epsilon_constraint_method(start_cost, end_cost, objective_rule, sense=minimize):
    epsilon_values = np.linspace(start_cost, end_cost, 10)[1:-1]  # Exclude start and end points
    results = []

    for epsilon in epsilon_values:
        # Remove old epsilon constraint if it exists
        if hasattr(model, 'epsilon_constraint'):
            model.del_component(model.epsilon_constraint)
        model.epsilon_constraint = Constraint(expr=(cost_rule(model) == epsilon))  # Fix cost at different levels

        # Remove old objective and set new one (Minimize Emissions)
        if hasattr(model, 'obj'):
            model.del_component(model.obj)
        model.obj = Objective(rule=objective_rule, sense=sense)

        solver.solve(model, tee=True)

        emissions = value(energy_rule(model))
        results.append({
            'Epsilon': epsilon,
            'Cost': value(cost_rule(model)),
            'Emissions': emissions,
            'Scales': {p: value(model.Scale[p]) for p in model.p},
            'Flows': {c: value(model.Flow[c]) for c in model.c}
        })

        model.display()
        # Clean up constraints for next iteration
        model.del_component(model.epsilon_constraint)
        model.del_component(model.obj)

    return results

# *Perform the epsilon constraint method for Min-Cost to Min-Emissions*
results = epsilon_constraint_method(min_cost, min_emissions_cost, energy_rule, sense=minimize)

# Convert to DataFrame
df_results_df = pd.DataFrame(results)
df_results_df = df_results_df[['Cost', 'Emissions']]

#  Create a new row with only 'Cost' and 'Emissions'
new_rows = [
    {'Cost': min_cost, 'Emissions': min_cost_emissions },
    {'Cost': min_emissions_cost,'Emissions': min_emissions}
]

# Append the new row to the DataFrame
df_results_df = pd.concat([ df_results_df, pd.DataFrame(new_rows)], ignore_index=True)
# You can now access or save the results as CSV
df_results_df.to_csv("epsilon_results_95_100.csv", index=False)

In [ ]:
import pandas as pd

df_90_2 =  pd.read_csv('epsilon_results_90_100.csv')
df_90_2.sort_values(by='Cost', ascending=False, inplace=True)

df_95 =  pd.read_csv('epsilon_results_95_100.csv')
df_95.sort_values(by='Cost', ascending=False, inplace=True)

df_90_1 =  pd.read_csv('epsilon_results_90_95.csv')
df_90_1.sort_values(by='Cost', ascending=False, inplace=True)

df_80 =  pd.read_csv('epsilon_results_80_90.csv')
df_80.sort_values(by='Cost', ascending=False, inplace=True)

df_70 =  pd.read_csv('epsilon_results_70_80.csv')
df_70.sort_values(by='Cost', ascending=False, inplace=True)

df_original =  pd.read_excel('Energy vs Profit vs Avg Recovery Rate Data for 84 LIB abstracts.xlsx', index_col=0).fillna(0)
df_original.sort_values(by='Profit (Cost of Recovered Materials)', ascending=False, inplace=True)
df_original.head()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset

fig, ax = plt.subplots()

# Plot lines and scatter points
ax.plot(df_95['Cost'], df_95['Emissions'], label= '(95-100)%', alpha =1.0)
ax.scatter(df_95['Cost'], df_95['Emissions'])

ax.plot(df_90_1['Cost'], df_90_1['Emissions'], label= '(90-95)%', alpha =1.0)
ax.scatter(df_90_1['Cost'], df_90_1['Emissions'])

ax.plot(df_80['Cost'], df_80['Emissions'], label= '(80-90)%', alpha =1.0)
ax.scatter(df_80['Cost'], df_80['Emissions'])

ax.plot(df_70['Cost'], df_70['Emissions'], label= '(70-80)%', alpha =1.0)
ax.scatter(df_70['Cost'], df_70['Emissions'])

# ax.scatter( df_original['Profit (Cost of Recovered Materials)'][:15], df_original['Energy'][:15], label= 'aa'  )

# Labels and legend
ax.legend(
    title='Recovery Efficiency of [Ni, Co, Li, Mn] for:',
    title_fontproperties={'weight': 'bold', 'size': 12},
    prop={'weight': 'bold', 'size': 12},
    loc='upper left',
    bbox_to_anchor=(0.00, 1.0),
    ncol=2,
    frameon=True  # optional: adds box around legend
)

# ax.set_xlim(1690, 1790)
# ax.set_ylim(0, 36000)

ax.set_xlabel("Profit [$]", fontsize=16, fontweight='bold')
ax.set_ylabel("Energy [kJoules]", fontsize=16, fontweight='bold')

# Bold ticks
ax.tick_params(axis='both', which='major', labelsize=10, width=1.8)
for spine in ax.spines.values():
    spine.set_linewidth(1.8)

# Grid
ax.grid(True, linestyle='--', alpha=0.7)

for label in ax.get_xticklabels():
    label.set_fontweight('bold')
for label in ax.get_yticklabels():
    label.set_fontweight('bold')

plt.savefig('pareto.pdf', dpi = 2000, bbox_inches= 'tight')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

# Color values based on profit
profit_values = df_original['Average Recovery Rate']

# Scatter plot with color mapping
scatter = plt.scatter(
    df_original['Profit (Cost of Recovered Materials)'],
    df_original['Energy'],
    c=profit_values,
    cmap='viridis',   # Try 'plasma', 'coolwarm', 'turbo' etc.
    edgecolor='black'
)

# Add colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('Recycling Efficiency (%)', fontsize=12, fontweight='bold')
cbar.ax.yaxis.set_tick_params(labelsize=10)
# cbar.ax.tick_params(labelweight='bold')  # This line is causing the error
cbar.ax.set_yticklabels(cbar.ax.get_yticklabels(), fontweight='bold') # Use set_yticklabels instead

# Add bold labels to each point
for i, row in df_original.iterrows():
    plt.text(
        row['Profit (Cost of Recovered Materials)'],
        row['Energy'],
        str(i),  # Replace 'i' with row['LabelColumn'] if available
        fontsize=8,
        fontweight='bold',
        ha='left',
        va='bottom'
    )

# Axis labels
plt.xlabel("Profit [$]", fontsize=14, fontweight='bold')
plt.ylabel("Energy [kJoules]", fontsize=14, fontweight='bold')

# Bold ticks
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

# Bold border lines
for spine in plt.gca().spines.values():
    spine.set_linewidth(1.8)

# Grid and show
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('/content/pareto_with_peaks.pdf', dpi = 2000, bbox_inches= 'tight')
plt.show()
